# The analysis of multiscale model results: a list of results

This notebook shows various analyses of the results of the multiscale model. We go into the details of the pulse shaping, the plasma profile, and we show also the build up of the harmonic signal.

See complementary analyses:
* [Jupyter notebook processing a list of results.](./analyse_cell_list.ipynb)
* [Jupyter notebook analysing a file, which contains also the outputs of TDSE module](../density_profile/analyse_density_profile.ipynb) (these results were removed in the actual tutorial because they are the largest datasets from the model).



## Load libraries & initial data

In [ ]:
## python modules used within this notebook
import numpy as np
from scipy import integrate
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib.animation
import matplotlib.colors as colors
import matplotlib.gridspec as gridspec
from contextlib import ExitStack
import os
import h5py
import sys
import copy
import MMA_administration as MMA
import mynumerics as mn
import units
from IPython.display import display, Markdown
from IPython.display import HTML

import dataformat_CUPRAD as dfC
import HHG

anims_author = 'Jan Vábek'

matplotlib.rcParams['animation.embed_limit'] = 200.
# print(matplotlib.rcParams['animation.embed_limit'])


# %%capture
# %matplotlib inline
# import mpld3
# mpld3.enable_notebook()

# %matplotlib agg
%matplotlib inline

Here we dicide if we `show` animations here or `save` them as well.

In [ ]:
visualisation = 'show'

## Load data

Compared to the [analysis of a single file](./analyse_cell_single.ipynb), we reduce the resolution in the radial coordinate $\rho$ to reduce the size of data within this notebook.

In [ ]:
# here we specify a coarser r-grid while loading the data
r_resolution_specifier = [False, 10e-6/3., 230e-6] # ['use the coarser data', radial step, maximal radial coordinate] (all in SI units)
list_of_loaded_simulations = ['results_cell_1.h5','results_cell_2.h5','results_cell_3.h5']
demos_path = os.path.join(os.environ['MULTISCALE_DEMOS'],'gas_cell')

h5files = [os.path.join(demos_path, foo) for foo in list_of_loaded_simulations] 

# load the data
Nsim = len(h5files)
CUPRAD_res = []
with ExitStack() as stack:
    # Open all files and store file objects in a list
    files = [stack.enter_context(h5py.File(h5file, 'r')) for h5file in h5files]
    
    # Load data from each file and append it to CUPRAD_res
    for f in files:
        CUPRAD_res.append(dfC.get_data(f,r_resolution=r_resolution_specifier))
        CUPRAD_res[-1].get_plasma(f,r_resolution=r_resolution_specifier)

zmax_all = np.min([foo.zgrid[-1] for foo in CUPRAD_res])

Here we print some basic characteristics of the simulation. We assume that the different simulations differs only in the discretisation in $z$ due to adaptive steps and all the other parameters are the same.

In [ ]:
# code to generate the following text
display(Markdown(
rf"""* The ($1/\mathrm{{e}}$) entry pulse duration ${
      CUPRAD_res[0].pulse_duration_entry   
      :.1f}~\mathrm{{fs}}$
* The box size is ($z-$ and $\rho$-grids start at 0):
    * $z_{{\mathrm{{max}}}}={
      1e3*zmax_all
      :.1f}~\mathrm{{mm}},~N_{{z,\text{{min}}}}={
      np.min([np.shape(foo.E_zrt)[0] for foo in CUPRAD_res])   
      :.0f},~N_{{z,\text{{max}}}}={
      np.max([np.shape(foo.E_zrt)[0] for foo in CUPRAD_res])   
      :.0f}$,
    * $t_{{\mathrm{{min/max}}}}=\mp{
      1e15*CUPRAD_res[0].tgrid[-1]   
      :.0f}~\mathrm{{fs}},~N_t={
      np.shape(CUPRAD_res[0].E_zrt)[2]   
      :.0f}$,
    * $\rho_{{\mathrm{{max}}}}={
      1e6*CUPRAD_res[0].rgrid[-1]   
      :.0f}~\mu\mathrm{{m}},~N_\rho={
      np.shape(CUPRAD_res[0].E_zrt)[1]   
      :.0f}.$
"""))

## Plot the propagating pulse
The visualisation is analogical to [the single-file case](./analyse_cell_single.ipynb), the difference is that the $z$-grids are different for each file. We thus change the approach and define first the $z$-values for plotting and find corresponiding subarrays. Next, we show only the electric fields here.

In [ ]:
tlim = np.asarray((-60,60))   # [fs]  the time range for plotting the propagating pulse
rlim = 300                    # [mum] the radial range for plotting the propagating pulse

dz_plot = 20e-6 # [m]
Ncolumns = 3 # this specifies into how many columns is the figure split

save_animation = (visualisation == 'save')
if save_animation:
    ani_outpath = os.path.join(os.environ['MULTISCALE_WORK_DIR'],'gas_cell', 'export')
    if not(os.path.exists(ani_outpath)): os.makedirs(ani_outpath)

In [ ]:
# Code to generate the animated figure

# set the number of points in z according to the spacing
Nz_plot = round(zmax_all/dz_plot)
# find indices according to tlim for all tgrids
k_t_min, k_t_max = tuple(zip(*[mn.FindInterval(1e15*CUPRAD_res[k1].tgrid,1.05*tlim) for k1 in range(len(CUPRAD_res))]))

Nrows = Nsim//Ncolumns if (Nsim%Ncolumns==0) else (Nsim//Ncolumns)+1

# Create figure

fig, axes = plt.subplots(Nrows, Ncolumns, figsize=(15, Nrows*4))  # Create a grid for plots
axes = axes.flatten()


pcs = []; cbars = []
# Create pcolormesh plots and colorbars
for k1, ax in enumerate(axes[:len(CUPRAD_res)]):
    
    # symmetrise and plot the data
    rgrid_sym, data_sym = mn.symmetrize_y(1e6*CUPRAD_res[k1].rgrid, 1e-9*CUPRAD_res[k1].E_zrt[0, :, k_t_min[k1]:k_t_max[k1]].T)
    pc = ax.pcolormesh(1e15*CUPRAD_res[k1].tgrid[k_t_min[k1]:k_t_max[k1]],
                       rgrid_sym,
                       data_sym.T,
                       shading='auto', cmap='seismic')
    pcs.append(pc)
    # make colorbar
    cbar = fig.colorbar(pc, ax=ax)
    cbar.ax.set_ylabel(r'$\mathcal{E}$ [GV/m]', rotation=90)
    cbars.append(cbar)    

    # Set axis properties
    ax.set_xlim(tlim)
    ax.set_title("z={:.2f} mm".format(1e3*CUPRAD_res[k1].zgrid[0]))
    ax.set_xlabel(r'$t~[\mathrm{fs}]$')
    ax.set_ylabel(r'$\rho~[\mu\mathrm{m}]$')

# turn off axes for empty plots
for k1 in range(Nsim,Ncolumns*Nrows): axes[k1].axis('off')



def update(frame):
    for k1 in range(len(CUPRAD_res)):
        kz_local = mn.FindInterval(CUPRAD_res[k1].zgrid,frame*dz_plot)
        # Update the data for each subplot
        data = 1e-9*CUPRAD_res[k1].E_zrt[kz_local, :, k_t_min[k1]:k_t_max[k1]]
        data_sym = mn.symmetrize_y(1e6*CUPRAD_res[k1].rgrid, 1e-9*CUPRAD_res[k1].E_zrt[kz_local, :, k_t_min[k1]:k_t_max[k1]].T)[1].T
        pcs[k1].set_array(data_sym.ravel())
        max_value_symmetric = np.max(np.abs((data.min(), data.max())))
        pcs[k1].set_clim(-max_value_symmetric,max_value_symmetric)
        cbars[k1].update_normal(pcs[k1])

        axes[k1].set_title("z={:.2f} mm".format(1e3*CUPRAD_res[k1].zgrid[kz_local]))

    return pcs

# Ensure the layout does not have overlaps and everything is nicely spaced
fig.tight_layout() 

# animate
ani = matplotlib.animation.FuncAnimation(fig,
                                         update,
                                         frames=Nz_plot,
                                         blit=True)

if save_animation:
    # Define the writer using ffmpeg for mp4 format and save it
    FFmpegWriter = matplotlib.animation.writers['ffmpeg']
    writer = FFmpegWriter(fps=fps, metadata=dict(artist=animation_author), bitrate=1800)

    ani.save(os.path.join(ani_outpath,'gas_cell_list_Efield.mp4'), writer=writer)

HTML(ani.to_jshtml())


## Plasma channels
Here we show plasma channels after the passage of the pulse.

In [ ]:
# Code to generate the figure

Nrows = Nsim//Ncolumns if (Nsim%Ncolumns==0) else (Nsim//Ncolumns)+1

fig, axes = plt.subplots(Nrows, Ncolumns, figsize=(15, Nrows*4))  # Create a 2x2 grid of subplots
axes = axes.flatten()


pcs = []; cbars = []
# Create pcolormesh plots and colorbars
for k1, ax in enumerate(axes[:len(CUPRAD_res)]):

    # symmetrize and plot the data
    rgrid_sym, data_sym = mn.symmetrize_y(1e6*CUPRAD_res[k1].plasma.rgrid, (1e2/CUPRAD_res[k1].effective_neutral_particle_density)*CUPRAD_res[k1].plasma.value_zrt[:, :, -1])
    pc = ax.pcolormesh(1e3*CUPRAD_res[k1].plasma.zgrid,
                       rgrid_sym,
                       data_sym.T,
                       shading='auto')

    pcs.append(pc)
    cbar = fig.colorbar(pc, ax=ax)
    cbar.ax.set_ylabel(r'relative plasma density [%]', rotation=90)
    cbars.append(cbar)    

    # Set axis properties
    ax.set_title(f"simulation {k1+1}")
    ax.set_xlabel(r'$z~[\mathrm{mm}]$')
    ax.set_ylabel(r'$\rho~[\mu\mathrm{m}]$')

# turn off axes for empty plots
for k1 in range(Nsim,Ncolumns*Nrows): axes[k1].axis('off')

fig.tight_layout() 
plt.show()

## XUV camera

Finally, we show the far field harmonic spectra and cummulative signal in the medium. The data are load here.

In [ ]:
# Set parameters
logscale = True
normalise_spectra = True
orders_to_plot = 3 

In [ ]:
# Code to generate the figure
Nrows = Nsim//Ncolumns if (Nsim%Ncolumns==0) else (Nsim//Ncolumns)+1

fig, axes = plt.subplots(Nrows, Ncolumns, figsize=(15, Nrows*4))  # Create a 2x2 grid of subplots
axes = axes.flatten()


pcs = []; cbars = []
# Create pcolormesh plots and colorbars
for k1, ax in enumerate(axes[:Nsim]):
    
    # read data from the respective file
    with h5py.File(h5files[k1], 'r') as f:
        far_field_signal =       f[MMA.paths['Hankel_outputs']+'/cumulative_field'][-1,:,:,0] +\
                              1j*f[MMA.paths['Hankel_outputs']+'/cumulative_field'][-1,:,:,1]
        theta_grid_Hankel  = f[MMA.paths['Hankel_outputs']+'/rgrid'][:]/mn.readscalardataset(f,MMA.paths['Hankel_inputs']+'/distance_FF','N')
        Hgrid_Hankel       = f[MMA.paths['Hankel_outputs']+'/ogrid'][:]/CUPRAD_res[k1].omega0

    # Symmetrize data and plot the data
    theta_grid_sym, data_sym = mn.symmetrize_y(1e3 * theta_grid_Hankel, np.abs(far_field_signal).T)
    scale_kwargs = {'norm' : colors.LogNorm(vmin=(10**(-orders_to_plot))*data_sym.max(), vmax=data_sym.max())} if logscale else {}
    pc = ax.pcolormesh(Hgrid_Hankel,
                       theta_grid_sym,
                       data_sym.T,
                       shading='auto',
                       **scale_kwargs)

    pcs.append(pc)
    cbar = fig.colorbar(pc, ax=ax, orientation = 'horizontal')
    cbar.ax.set_xlabel(r'harmonic signal [arb. .u.]')
    cbars.append(cbar)
    
    # Set axis properties, labels, ...
    ax.set_title(f"simulation {k1 + 1}")
    ax.set_xlabel('harmonic order [-]')
    H_min, H_max = ax.get_xlim() 
    ax.set_xticks(range(int(np.ceil(H_min)) | 1, int(np.floor(H_max)) + 1, 10))              # Generate major odd ticks within the H-range
    ax.set_xticks(range(int(np.ceil(H_min)) | 1, int(np.floor(H_max)) + 1, 2), minor = True) # Generate odd ticks within the H-range
    ax.set_ylabel(r'divergence [mrad]')

# turn off axes for empty plots
for k1 in range(Nsim,Ncolumns*Nrows): axes[k1].axis('off')


# Ensure the layout does not have overlaps and everything is nicely spaced
fig.tight_layout() 
plt.show()